In [1]:
"""
01, encoding, 30, 0.30
02, encoding, 64, 0.31596
03, dummy2 (xgb for each col), 30, 0.26101
04, dummy3 (raw + xgb for all col), 30, 0.301124
"""

'\n01, raw xgb + raw RF, simple average, 0.378522\n02, raw xgb + raw RF, rank average, 0.377719\n03, raw xgb, 0.381522 (works fine!)\n04, raw RF, 0.365994\n05, raw xgb + RF, rank multiplication, 0.380349 (works better than I thought)\n06, should i try rank min, 0.380458\n07, stack RF, Extra, XGB, 0.382054\n08, xgb (subsample 0.8), 0.382557\n'

In [2]:
"""
categorical encoding + knn
xgb features + knn
categorical only + various estimator
numerical only + various estimator
variable selection (xgb, RF) + various estimator
numerical as categorical
second order features
standardize kfold transformer
numerical as categorical + 2nd order + linear
how to incorporate the info T1 vs T2 into modeling?
"""

'\ncategorical encoding + knn\nxgb features + knn\ncategorical only + various estimator\nnumerical only + various estimator\nvariable selection (xgb, RF) + various estimator\nnumerical as categorical\nsecond order features\nstandardize kfold transformer\nnumerical as categorical + 2nd order + linear\n'

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train_set = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

data = pd.concat([train, test], axis=0).reset_index()

## --------------------- Feature Start -----------------------

In [3]:
# start
categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Id'])
numerical = list(numerical)

In [50]:
# raw feature
from sklearn.preprocessing import StandardScaler
import re

SS = StandardScaler()
feat_raw = data[numerical].applymap(float)
feat_raw[numerical] = SS.fit_transform(feat_raw[numerical])
feat_raw_cat = pd.get_dummies(data[categorical])

feat_raw = pd.concat([feat_raw, feat_raw_cat], axis=1)
feat_raw.shape

(101999, 111)

In [74]:
%%time
# dummy 3

from xgboost import XGBRegressor
import xgboost as xgb

feat_dummy = pd.DataFrame()


dTmp = xgb.DMatrix(feat_raw[train_set==1], label=train.Hazard[train_set==1])
reg = xgb.train({'max_depth':7, 'min_child_weight':100, 'objective':'reg:linear'}, dTmp, 3)

dTmp = xgb.DMatrix(feat_raw)
feat_dummy = reg.predict(dTmp, pred_leaf=True)
feat_dummy = pd.DataFrame(feat_dummy, columns=['dummy_'+str(i) for i in range(3)])

for x in feat_dummy.columns:
    feat_dummy[x] = feat_dummy[x].astype('category')    

feat_dummy = pd.get_dummies(feat_dummy)
print(feat_dummy.shape)

(101999, 168)
CPU times: user 3.94 s, sys: 88 ms, total: 4.02 s
Wall time: 3.54 s


In [75]:
# assemble features

tmp = pd.concat([feat_raw, feat_dummy], axis=1)
train_c = tmp[train_set==1]
test_c = tmp[train_set==0]

print(train_c.shape)
print(test_c.shape)

(50999, 279)
(51000, 279)


## --------------------- Modeling -----------------------

In [76]:
%%time
# knn 10
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import cross_val_score
from wrappers import cross_validate

knn = KNeighborsRegressor(30)
score = cross_validate(knn, train_c, train.Hazard, nfold=5)
print(score)

0.301124301553
CPU times: user 17min 43s, sys: 188 ms, total: 17min 44s
Wall time: 17min 39s


In [44]:
pd.DataFrame({'Id': test.Id, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/ensemble_07.csv', index=0)

## --------------------- Test Ground -----------------------

## --------------------- Factory -----------------------

In [10]:
# categorical encoding
from wrappers import Whitener
from sklearn.preprocessing import StandardScaler

feat_encode = data[categorical].copy()
for cat in categorical:
    hot_deck = data.groupby(cat).Hazard.mean().to_dict()
    feat_encode[cat] = feat_encode[cat].apply(lambda x: hot_deck[x])

feat_encode = feat_encode.rename(columns=lambda x: 'encode_'+x)
feat_encode = pd.concat([feat_encode, data[numerical]], axis=1)

# feat_encode[feat_encode.columns] = StandardScaler().fit_transform(feat_encode)
feat_encode[feat_encode.columns] = Whitener().fit_transform(feat_encode)

In [ ]:
%%time
# dummy 2

from xgboost import XGBRegressor
import xgboost as xgb

feat_dummy = pd.DataFrame()

for x in numerical:
    dTmp = xgb.DMatrix(data.ix[train_set==1, [x]], label=train.Hazard[train_set==1])
    reg = xgb.train({'max_depth':100, 'min_child_weight':30, 'objective':'reg:linear'}, dTmp, 1)
    
    dTmp = xgb.DMatrix(data[[x]], label=train.Hazard)
    feat_dummy['dummy_'+x] = reg.predict(dTmp, pred_leaf=True)
    feat_dummy['dummy_'+x] = feat_dummy['dummy_'+x].astype('category')
    
feat_dummy = pd.concat([feat_dummy, data[categorical]], axis=1)
feat_dummy = pd.get_dummies(feat_dummy)

In [47]:
# dummy 
feat_dummy = data[numerical+categorical].copy()
for x in feat_dummy.columns:
    feat_dummy[x] = feat_dummy[x].astype('category')
    
feat_dummy = pd.get_dummies(feat_dummy)

In [18]:
%%time
# sofia feature
from sofia_utils import sofia_kmeans

n_clusters = 100

sofia = sofia_kmeans(n_clusters=n_clusters, iterations=1000, mapping_threshold=0.0001)

feat_sofia = sofia.fit_transform(data[numerical])

feature_names = ['sofia_'+str(i+1) for i in range(n_clusters)]
feat_sofia = pd.DataFrame(feat_sofia, columns=feature_names)
print(feat_sofia.shape)

from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler()
feat_sofia.ix[:, :] = MMS.fit_transform(feat_sofia.values)

extract percent:  853192.0
sparcity:  0.695033872881
(101999, 100)
CPU times: user 6.6 s, sys: 1.11 s, total: 7.71 s
Wall time: 14.5 s


In [6]:
%%time
from sklearn.preprocessing import PolynomialFeatures

PF = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
train_c = PF.fit_transform(train_c)
print(train_c.shape)

(50999, 6216)
CPU times: user 6.03 s, sys: 2.56 s, total: 8.59 s
Wall time: 8.59 s


In [75]:
%%time
# kmeans features

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

kmeans = KMeans(80, random_state=12345, n_init=1)
feat_kmeans_train = kmeans.fit_transform(SS.fit_transform(train[numerical]))
feat_kmeans_test = kmeans.transform(SS.transform(test[numerical]))

# print('variance lost:')
# print(kmeans.inertia_ / KMeans(1, random_state=12345, n_init=1).fit(train[numerical]).inertia_)

feat_kmeans_train = pd.DataFrame(feat_kmeans_train).add_prefix('kmeans_')
feat_kmeans_test = pd.DataFrame(feat_kmeans_test).add_prefix('kmeans_')

numerical += feat_kmeans_train.columns.values.tolist()
train_c = pd.concat([train_c, feat_kmeans_train], axis=1)
test_c = pd.concat([test_c, feat_kmeans_test], axis=1)

CPU times: user 14.5 s, sys: 4 ms, total: 14.5 s
Wall time: 14.5 s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [49]:
# count features

feat_count_train = train[categorical].copy()
feat_count_test = test[categorical].copy()

for cat in categorical:
    numerical.append('cnt_'+cat)
    hotDeck = train[cat].value_counts().reset_index().rename(columns={'index': cat, 0: 'cnt_'+cat})
    feat_count_train = pd.merge(feat_count_train, hotDeck, on=cat)
    del feat_count_train[cat]
    
    feat_count_test = pd.merge(feat_count_test, hotDeck, on=cat)
    del feat_count_test[cat]
    
train_c = pd.concat([train_c, feat_count_train], axis=1)
test_c = pd.concat([test_c, feat_count_test], axis=1)

In [81]:
%%time
# reduce
from xgboost import XGBRegressor
import xgboost as xgb

feat_reduce_train = pd.DataFrame()
feat_reduce_test = pd.DataFrame()

for num in numerical:
    dTmp = xgb.DMatrix(train[[num]], label=train.Hazard)
    reg = xgb.train({'max_depth':8, 'min_child_weight':30, 'objective':'reg:linear'}, dTmp, 1)
    
    feat_reduce_train['discrete_'+num] = reg.predict(dTmp, pred_leaf=True)
    feat_reduce_train['discrete_'+num] = feat_reduce_train['discrete_'+num].map(str)
    
    feat_reduce_test['discrete_'+num] = reg.predict(xgb.DMatrix(test[[num]]), pred_leaf=True)
    feat_reduce_test['discrete_'+num] = feat_reduce_test['discrete_'+num].map(str)

Wall time: 6.86 s


In [73]:
# discrete

feat_discrete_train = train[numerical].applymap(str)
feat_discrete_test = test[numerical].applymap(str)

In [88]:
# combine all features
cTrain = train[numerical+categorical]
cTest = test[numerical+categorical]

# cTrain = pd.concat([train[numerical+categorical], feat_reduce_train], axis=1)
# cTest = pd.concat([test[numerical+categorical], feat_reduce_test], axis=1)